### Importación de librerías

In [ ]:
import os
import cv2
import numpy as np
import pickle
import mediapipe as mp
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

### Definición de variables

In [ ]:
DATASET_PATH = "../../dataset/gestos_manos" 
MODEL_FILE = "modelo_manos_red_augmentation.pkl"

### Lógica de entreno

In [ ]:
# Etiquetas
MAPA_ETIQUETAS = {
    "0_Neutro": 0,
    "1_Mano_Der_Arriba": 1,
    "2_Mano_Izq_Arriba": 2,
    "3_Punos_Cerrados": 3,
    "4_Pulgar_Arriba": 4,
    "5_Victoria": 5,
    "6_Rock": 6,
    "7_Llamada": 7,
    "8_Ok": 8
}

# --- CONFIGURACIÓN MEDIAPIPE HANDS ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,       
    min_detection_confidence=0.5
)

# ==========================================
#           FUNCIONES MATEMÁTICAS
# ==========================================

def normalizar_puntos(landmarks):
    """Centra y escala los puntos de la mano."""
    coords = np.array([[lm.x, lm.y] for lm in landmarks])
    
    # 1. Centrar
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    
    # 2. Escalar
    max_dist = np.max(np.abs(centered))
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
        
    return normalized.flatten()

def aumentar_datos(vector_original):
    """Genera variaciones: Ruido, Escala y Rotación."""
    variaciones = []
    variaciones.append(vector_original)
    
    puntos_2d = vector_original.reshape(-1, 2)

    # 1. Ruido ligero
    ruido = np.random.normal(0, 0.01, puntos_2d.shape)
    variaciones.append((puntos_2d + ruido).flatten())

    # 2. Escala (Zoom in/out leve)
    factor_escala = np.random.uniform(0.95, 1.05)
    variaciones.append((puntos_2d * factor_escala).flatten())
    
    # 3. Rotación (+/- 15 grados)
    theta = np.radians(np.random.uniform(-15, 15))
    c, s = np.cos(theta), np.sin(theta)
    matriz_rotacion = np.array(((c, -s), (s, c)))
    puntos_rotados = np.dot(puntos_2d, matriz_rotacion)
    variaciones.append(puntos_rotados.flatten())

    return variaciones

# ==========================================
#           PASO 1: AUDITORÍA
# ==========================================
def auditar_dataset():
    print(f"\n{'='*40}")
    print(f"PASO 1: AUDITORÍA DEL DATASET (MANOS)")
    print(f"{'='*40}")
    
    if not os.path.exists(DATASET_PATH):
        print(f"ERROR CRÍTICO: La ruta '{DATASET_PATH}' no existe.")
        return False

    carpetas = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    total_global_ok = 0
    archivos_fallidos = []

    for carpeta in carpetas:
        if carpeta not in MAPA_ETIQUETAS:
            continue 
        ruta_carpeta = os.path.join(DATASET_PATH, carpeta)
        imagenes = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        count_ok = 0
        count_fail = 0

        print(f"Carpeta '{carpeta}': {len(imagenes)} imágenes.")

        for archivo in imagenes:
            ruta_img = os.path.join(ruta_carpeta, archivo)
            img = cv2.imread(ruta_img)

            if img is None:
                count_fail += 1
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)

            # VERIFICACIÓN ESPECÍFICA PARA MANOS
            if results.multi_hand_landmarks:
                count_ok += 1
            else:
                count_fail += 1
                archivos_fallidos.append(os.path.join(carpeta, archivo))

        print(f"   -> Detectadas: {count_ok} | No detectadas: {count_fail}")
        total_global_ok += count_ok

    print("-" * 30)
    if archivos_fallidos:
        print(f"Se encontraron {len(archivos_fallidos)} imágenes donde NO se ven manos (se ignorarán).")
    else:
        print("Dataset limpio. Todas las manos detectadas.")

    return total_global_ok > 0

# ==========================================
#           PASO 2: ENTRENAMIENTO
# ==========================================
def entrenar_modelo():
    print(f"\n{'='*40}")
    print(f"PASO 2: ENTRENAMIENTO (+ DATA AUGMENTATION)")
    print(f"{'='*40}")

    data = []
    labels = []

    for nombre_carpeta, etiqueta_num in MAPA_ETIQUETAS.items():
        ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
        if not os.path.exists(ruta_carpeta): 
            print(f"⚠️ La carpeta {nombre_carpeta} no existe, saltando...")
            continue
        
        archivos = os.listdir(ruta_carpeta)
        print(f"Procesando clase '{nombre_carpeta}'...")
        
        muestras_clase = 0
        for archivo in archivos:
            if not (archivo.lower().endswith(('.jpg', '.png', '.jpeg'))): continue
            
            img = cv2.imread(os.path.join(ruta_carpeta, archivo))
            if img is None: continue
            
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)
            
            if results.multi_hand_landmarks:
                # Tomamos solo la primera mano detectada (índice 0)
                hand_landmarks = results.multi_hand_landmarks[0]
                
                # 1. Normalizar
                vector_base = normalizar_puntos(hand_landmarks.landmark)
                
                # 2. Aumentar datos (Original + 3 variaciones = 4 muestras por foto)
                vectores_aumentados = aumentar_datos(vector_base)
                
                # 3. Guardar
                for v in vectores_aumentados:
                    data.append(v)
                    labels.append(etiqueta_num)
                    muestras_clase += 1
        
        print(f"   -> Generados {muestras_clase} vectores de características.")

    # --- ENTRENAMIENTO ---
    data = np.array(data)
    labels = np.array(labels)

    if len(data) == 0:
        print("ERROR: No se generaron datos. Revisa el dataset.")
        return

    print(f"\nTotal de muestras para entrenamiento: {len(data)}")
    print("Entrenando Red Neuronal (MLP Classifier)...")

    # Configuración del modelo (ajustada para manos)
    model = MLPClassifier(
        hidden_layer_sizes=(64, 32), # Capas un poco más ligeras que para cara, suele bastar
        activation='relu', 
        solver='adam', 
        max_iter=1500,
        random_state=42
    )

    # Validación cruzada
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, data, labels, cv=kfold)
    
    print(f"Precisión estimada (Cross-Validation): {scores.mean() * 100:.2f}%")

    # Entrenamiento final
    model.fit(data, labels)
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"\n¡ÉXITO! Modelo de manos guardado en: {MODEL_FILE}")

# ==========================================
#           EJECUCIÓN PRINCIPAL
# ==========================================

if __name__ == "__main__":
    # 1. Auditar
    dataset_valido = auditar_dataset()

    # 2. Entrenar si la auditoría pasa
    if dataset_valido:
        entrenar_modelo()
    else:
        print("\nDETENIDO: No hay imágenes válidas para entrenar.")